In [1]:
# SOURCE:  https://github.com/atreadw1492/yahoo_fin/blob/master/yahoo_fin/stock_info.py

In [2]:
# finviz:  https://www.youtube.com/watch?v=cGL1FJKEfAk

# Other examples
# https://www.youtube.com/watch?v=Ibgk3L96kco

In [3]:
# DOCU:   https://theautomatic.net/yahoo_fin-documentation/

## Install libraries

In [4]:
# !pip install yahoo_fin

In [5]:
# !pip install pandas_datareader

## Import libraries

In [6]:
# this are from my .py files
import yahoo_finance_stocks as yf
import pandas_datareader_stocks as datareader

In [7]:
import datetime as dt
from datetime import datetime
import pandas as pd

## Correlation SPY AND VIX

In [8]:
# I want to validate when VIX and SPY open negative, so SPY close positive

In [9]:
start = dt.datetime(2022, 1, 1)
end   = dt.datetime.today()  # almost today

In [10]:
cols_print = ['Date','prev_close','chg%_premarket', 'Open', 'Close']

df_vix = datareader.get_cleaned_data_ticker('^VIX',start,end)
df_vix = df_vix[cols_print]

df_vix.tail(3)

,Date,prev_close,chg%_premarket,Open,Close
200,2022-10-19,30.50,1.422,30.940001,30.760000
201,2022-10-20,30.76,1.725,31.299999,29.980000
202,2022-10-21,29.98,0.761,30.209999,29.690001


In [11]:
cols_print = ['Date','prev_close', 'chg%_premarket', 'open', 'close', 'volume','chg%_intraday']

df_spy = yf.get_cleaned_data_startdate("SPY", start)
df_spy = df_spy[cols_print]


## esta es una funcion nativa
#df_spy['Chg%_intraday2'] = 100*df_spy[['open','close']].pct_change(axis=1)['close'].round(5)
df_spy.tail(3)

,Date,prev_close,chg%_premarket,open,close,volume,chg%_intraday
200,2022-10-19,371.130005,-0.580,368.989990,368.500000,79746900,-0.133
201,2022-10-20,368.500000,-0.128,368.029999,365.410004,88283100,-0.717
202,2022-10-21,365.410004,-0.079,365.119995,374.290009,130897700,2.450


In [12]:
#. join both dataframes

In [13]:
df_join_vix_spy = pd.merge(
    df_vix,
    df_spy,
    how="inner",
    on="Date",
    suffixes=("_vix", "_spy")
)

In [14]:
cols=["Date","chg%_premarket_vix","chg%_premarket_spy","chg%_intraday"]
df_vix_spy = df_join_vix_spy[cols]

df_join_vix_spy.tail(40)

In [15]:
def filter_df_vix_spy(df, filter1, filter2):
    return df.where(filter1 & filter2).dropna()

def count_stats_vix_spy(df, condition, spy_close_positive=True):
    
    if spy_close_positive:
        filterSpyIntraday = df["chg%_intraday"] > 0
    else:
        filterSpyIntraday = df["chg%_intraday"] < 0
        
    df2 = df.where(filterSpyIntraday).dropna()
    df2["condition_premartek&_postmarket"] = condition
    
    return df2

In [16]:
## vix and spy negative
filter_vix_neg = df_vix_spy["chg%_premarket_vix"] < 0 
filter_spy_neg = df_vix_spy["chg%_premarket_spy"] < 0

## vix and spy positive
filter_vix_pos = df_vix_spy["chg%_premarket_vix"] > 0
filter_spy_pos = df_vix_spy["chg%_premarket_spy"] > 0

## vix almost negative and spy negative
filter_vix_almostneg = df_vix_spy["chg%_premarket_vix"] < 1 ## almost negative



## DF: filter both negative and both positive
df_vix_spy_both_neg = filter_df_vix_spy(df_vix_spy,filter_vix_neg,filter_spy_neg)
df_vix_spy_both_pos = filter_df_vix_spy(df_vix_spy,filter_vix_pos,filter_spy_pos)


df_vix_spy_both_neg1 = filter_df_vix_spy(df_vix_spy,filter_vix_neg,filter_spy_pos)
df_vix_spy_both_pos1 = filter_df_vix_spy(df_vix_spy,filter_vix_pos,filter_spy_neg)


df_vix_spy_both_neg2 = filter_df_vix_spy(df_vix_spy,filter_vix_almostneg,filter_spy_neg)



In [17]:
df_stats1 = count_stats_vix_spy(df_vix_spy_both_neg, "both_neg_&_spy_close_neg", spy_close_positive=False)
df_stats2 = count_stats_vix_spy(df_vix_spy_both_neg, "both_neg_&_spy_close_pos", spy_close_positive=True)

df_stats3 = count_stats_vix_spy(df_vix_spy_both_pos, "both_pos_&_spy_close_neg", spy_close_positive=False)
df_stats4 = count_stats_vix_spy(df_vix_spy_both_pos, "both_pos_&_spy_close_pos", spy_close_positive=True)

In [18]:
df_stats5 = count_stats_vix_spy(df_vix_spy_both_neg1, "vix_neg_spy_pos_&_spy_close_neg", spy_close_positive=False)
df_stats6 = count_stats_vix_spy(df_vix_spy_both_neg1, "vix_neg_spy_pos_&_spy_close_pos", spy_close_positive=True)

df_stats7 = count_stats_vix_spy(df_vix_spy_both_pos1, "vix_pos_spy_neg_&_spy_close_neg", spy_close_positive=False)
df_stats8 = count_stats_vix_spy(df_vix_spy_both_pos1, "vix_pos_spy_neg_&_spy_close_pos", spy_close_positive=True)

In [19]:
df_stats9 = count_stats_vix_spy(df_vix_spy_both_neg2, "both_almostneg_&_spy_close_pos", spy_close_positive=True)

In [20]:
data = [df_stats1, df_stats2, df_stats3, df_stats4, df_stats5, df_stats6, df_stats7, df_stats8, df_stats9]
df_stats = pd.concat(data, ignore_index=True, sort=False)

In [21]:
df_stats_count = df_stats.groupby(['condition_premartek&_postmarket'])['condition_premartek&_postmarket'].count()
df_stats_count

condition_premartek&_postmarket
both_almostneg_&_spy_close_pos     21
both_neg_&_spy_close_neg           10
both_neg_&_spy_close_pos            9
both_pos_&_spy_close_neg           20
both_pos_&_spy_close_pos           21
vix_neg_spy_pos_&_spy_close_neg    24
vix_neg_spy_pos_&_spy_close_pos    26
vix_pos_spy_neg_&_spy_close_neg    44
vix_pos_spy_neg_&_spy_close_pos    46
Name: condition_premartek&_postmarket, dtype: int64

## Get All Stock Symbols

## Financial: Income, Balance and Cash flow

In [22]:
yearly_json = yf.get_financials("AAPL",True,False) # yearly
quarterly_json = yf.get_financials("AAPL",False,True) # quarterly

In [23]:
temp1 = quarterly_json["quarterly_income_statement"]
temp2 = quarterly_json["quarterly_balance_sheet"]
temp3 = quarterly_json["quarterly_cash_flow"]

In [24]:
##add_thousand_comma_separator(temp1,"2021-12-25")

In [25]:
temp1 = yf.convert_dataframe_reset_index(temp1)
temp11 = yf.rename_columns_dataframe(temp1)
temp11_1 = yf.convert_quarters_to_currency_format(temp11)
temp11_1

,Breakdown,2022Q2,2022Q1,2021Q4,2021Q3
0,researchDevelopment,"$6,797,000,000","$6,387,000,000","$6,306,000,000","$5,772,000,000"
1,effectOfAccountingCharges,$0,$0,$0,$0
2,incomeBeforeTax,"$23,066,000,000","$30,139,000,000","$41,241,000,000","$23,248,000,000"
3,minorityInterest,$0,$0,$0,$0
4,netIncome,"$19,442,000,000","$25,010,000,000","$34,630,000,000","$20,551,000,000"
5,sellingGeneralAdministrative,"$6,012,000,000","$6,193,000,000","$6,449,000,000","$5,616,000,000"
6,grossProfit,"$35,885,000,000","$42,559,000,000","$54,243,000,000","$35,174,000,000"
7,ebit,"$23,076,000,000","$29,979,000,000","$41,488,000,000","$23,786,000,000"
8,operatingIncome,"$23,076,000,000","$29,979,000,000","$41,488,000,000","$23,786,000,000"
9,otherOperatingExpenses,$0,$0,$0,$0


In [26]:
temp11.dtypes

Breakdown    object
2022Q2       object
2022Q1       object
2021Q4       object
2021Q3       object
dtype: object

## Get Earnings History

In [27]:
ticker = "TSLA"
start = dt.datetime(2010, 1, 1)

In [28]:
df_ticker = yf.prepare_data_for_earnings(ticker, start)

df_ticker

,Date,yesterday_chg%_intraday,chg%_premarket,today_chg%_intraday,tomorrow_chg%_intraday
0,2010-06-29,NaN,-0.000,20.469,-8.225
1,2010-06-30,20.469,7.367,-8.225,-13.843
2,2010-07-01,-8.225,4.680,-13.843,-19.792
3,2010-07-02,-13.843,4.522,-19.792,-24.146
4,2010-07-06,-19.792,4.000,-24.146,-3.797
...,...,...,...,...,...
3097,2022-10-17,-9.278,2.404,4.244,-4.228
3098,2022-10-18,4.244,4.423,-4.228,1.009
3099,2022-10-19,-4.228,-0.177,1.009,-0.482
3100,2022-10-20,1.009,-6.606,-0.482,3.740


In [29]:
cols = ["ticker","Date_EPS","Time Report", "epsestimate","epsactual","epssurprisepct"]

df_EPS = yf.get_processed_earnings_history(ticker)[cols]

#df_EPS = df_EPS[cols]

In [31]:
df_EPS.dtypes

ticker                    object
Date_EPS          datetime64[ns]
Time Report               object
epsestimate              float64
epsactual                float64
epssurprisepct           float64
dtype: object

In [32]:
df_join_ticker_eps = pd.merge(
    df_EPS, df_ticker,
    how="inner",
    left_on="Date_EPS",
    right_on="Date"
)

df_join_ticker_eps.drop(columns=["Date"],inplace=True)

In [33]:
df_join_ticker_eps

,ticker,Date_EPS,Time Report,epsestimate,epsactual,epssurprisepct,yesterday_chg%_intraday,chg%_premarket,today_chg%_intraday,tomorrow_chg%_intraday
0,TSLA,2022-10-19,AMC,0.99,1.05,5.74,-4.228,-0.177,1.009,-0.482
1,TSLA,2022-07-20,AMC,0.61,0.76,25.12,0.216,0.508,0.290,6.110
2,TSLA,2022-04-20,AMC,0.75,1.07,42.50,2.246,0.180,-5.403,-6.538
3,TSLA,2022-01-26,AMC,0.79,0.85,7.22,0.457,3.573,-1.602,-12.575
4,TSLA,2021-10-20,AMC,0.53,0.62,16.76,-1.534,0.125,0.052,4.251
5,TSLA,2021-07-26,AMC,0.33,0.48,47.71,-0.463,1.166,1.011,-2.888
6,TSLA,2021-04-26,NaN,0.26,0.31,17.87,1.316,1.565,-0.379,-1.876
7,TSLA,2021-01-27,AMC,0.34,0.27,-22.38,-0.939,-1.464,-0.716,1.847
8,TSLA,2020-10-21,AMC,0.19,0.25,30.41,-2.325,0.180,-0.014,-3.788
9,TSLA,2020-07-22,AMC,-0.04,0.44,1278.38,-4.563,1.916,-0.419,-10.963
